## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head(4)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Rikitea,-23.1203,-134.9692,296.64,71,52,6.39,PF,2021-09-16 23:43:02,light rain
1,1,Cape+Town,-33.9258,18.4232,286.42,71,0,1.79,ZA,2021-09-16 23:44:02,clear sky
2,2,Mataura,-46.1927,168.8643,280.64,51,32,2.74,NZ,2021-09-16 23:46:29,scattered clouds
3,3,Victoria,22.2855,114.1577,302.22,82,52,0.45,HK,2021-09-16 23:51:16,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 290
What is the maximum temperature you would like for your trip? 300


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df= city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Rikitea,-23.1203,-134.9692,296.64,71,52,6.39,PF,2021-09-16 23:43:02,light rain
4,4,Mahebourg,-20.4081,57.7000,294.31,94,75,1.54,MU,2021-09-16 23:46:20,broken clouds
8,8,Pirgos,35.1167,25.7178,298.42,53,0,3.91,GR,2021-09-16 23:47:11,clear sky
9,9,Atuona,-9.8000,-139.0333,298.28,79,18,7.11,PF,2021-09-16 23:47:28,light rain
10,10,Qui+Nhon,13.7667,109.2333,299.68,86,75,1.38,VN,2021-09-16 23:51:25,broken clouds
11,11,Gambela,8.2500,34.5833,294.09,97,100,1.59,ET,2021-09-16 23:52:41,moderate rain
12,12,Tura,25.5198,90.2201,295.29,95,99,1.41,IN,2021-09-16 23:50:46,overcast clouds
16,16,Sokoto,13.0609,5.2390,297.16,82,64,5.33,NG,2021-09-16 23:52:43,broken clouds
29,29,Mareeba,-17.0000,145.4333,297.86,63,29,4.08,AU,2021-09-16 23:50:10,scattered clouds
30,30,Pisco,-13.7000,-76.2167,290.18,77,20,6.17,PE,2021-09-16 23:50:22,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City_ID        306
City           306
Lat            306
Lng            306
Max Temp       306
Humidity       306
Cloudiness     306
Wind Speed     306
Country        305
Date           306
Description    306
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_df= preferred_city_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,296.64,light rain,-23.1203,-134.9692,
4,Mahebourg,MU,294.31,broken clouds,-20.4081,57.7000,
8,Pirgos,GR,298.42,clear sky,35.1167,25.7178,
9,Atuona,PF,298.28,light rain,-9.8000,-139.0333,
10,Qui+Nhon,VN,299.68,broken clouds,13.7667,109.2333,
11,Gambela,ET,294.09,moderate rain,8.2500,34.5833,
12,Tura,IN,295.29,overcast clouds,25.5198,90.2201,
16,Sokoto,NG,297.16,broken clouds,13.0609,5.2390,
29,Mareeba,AU,297.86,scattered clouds,-17.0000,145.4333,
30,Pisco,PE,290.18,few clouds,-13.7000,-76.2167,


In [8]:
final_hotels = []
from tqdm import tqdm

for index, data in tqdm(hotel_df.iterrows()):
    nearby_hotels = []
    lat = str(data['Lat'])
    lng = str(data['Lng'])
   
    # Set the parameters to search for a hotel in Paris.
    params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key,
        "location": lat+','+lng}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    keys = hotels.keys()
    for key in keys:
        if key == 'results':
            dat = hotels[key]
            for d in dat:
                nearby_hotels.append(d['name'])
    if len(nearby_hotels) <= 0:
        nearby_hotels.append('')
        
    final_hotels.append(nearby_hotels[0])
    
print('Len of hotels',len(hotels))

305it [01:37,  3.13it/s]

Len of hotels 4


In [9]:
len(final_hotels)

305

In [10]:
hotel_df.shape

(305, 7)

In [11]:
final_hotels

['People ThankYou',
 'Shandrani Beachcomber Resort & Spa',
 'Daios Cove',
 'Villa Enata',
 'Hoang Yen Canary Hotel',
 'Solomon Debebe Hotel',
 'Hotel RIKMAN Continental',
 'Shukura Coral Hotel',
 'Trinity Plains Tourist Park',
 'Hotel San Isidro Oficial',
 'Hotel do Campo',
 'املاك خالد الشعشوع',
 'Laurel Inn & Conference Center',
 'Embassy Suites by Hilton Portland Downtown',
 'Casa Das Faias',
 'Holiday Inn Express Lompoc, an IHG Hotel',
 'Pousada Porto Praia',
 'Pousada Monte Carmelo O Pão',
 'Quinta da Meia Eira',
 'Posada Villaven C.A.',
 'Gran Hotel Mar del Plata',
 'Chambres d\'hôte "La Trinité"',
 'Marion',
 '',
 'Resort Terra Paraiso',
 'Mutsu Park Hotel',
 'Hotel Xaguate',
 'Finch Bay Galapagos Hotel',
 'Villa Oasis',
 'Sunset Mobile Home Park',
 'Paradise Inn',
 'Hotel Mirage',
 '味処・民宿 まつや',
 'Shanti Maurice Resort & Spa',
 'Welcomhotel By ITC Hotels, Bay Island, Port Blair',
 'Screensaver',
 'Hotel Pigalle',
 'Courtyard by Marriott Danbury',
 'Ikathouse',
 '',
 'Royal Park 

In [12]:
hotel_df['Hotel Name'] = final_hotels

In [13]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,296.64,light rain,-23.1203,-134.9692,People ThankYou
4,Mahebourg,MU,294.31,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
8,Pirgos,GR,298.42,clear sky,35.1167,25.7178,Daios Cove
9,Atuona,PF,298.28,light rain,-9.8000,-139.0333,Villa Enata
10,Qui+Nhon,VN,299.68,broken clouds,13.7667,109.2333,Hoang Yen Canary Hotel
...,...,...,...,...,...,...,...
671,Pokhara,NP,292.24,overcast clouds,28.2333,83.9833,Butterfly Lodge
672,Ponta+Delgada,PT,294.55,few clouds,37.7333,-25.6667,Vila Nova Hotel
673,Fuyu,CN,290.20,clear sky,45.1833,124.8167,Education Hostel
675,Pulandian,CN,293.22,light rain,39.3953,121.9669,HR East Hotel


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('Hotel_Data.csv',index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 

# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))